In [4]:
import os
import csv
import datetime


#将时间戳转化为时间
def covertTime(stamp):
    dateArray = datetime.datetime.utcfromtimestamp(stamp)
    otherStyleTime = dateArray.strftime("%Y-%m-%d %H:%M:%S")
    return otherStyleTime

#通过 key 查找到对应的 value
def findKeyValue(data, key):
    for d in data:
        if(d.find(key)!=-1):
            value = d.split('=')[1]
            if(key == 'stamp'):
                value = covertTime(int(value)/1000)
            
            return value

#根据 title 将数据拆分到数组中
def splitData(data, title):
    data_re = data.split('?')[1]
    data_re_1 = data_re.split('&')
    arr = []
    for key in title:
        arr.append(findKeyValue(data_re_1, key))
    return arr

def splitLine(line, title):
    if(line.find('app_package_name=com.sogou.translatorpen') != -1):
            splits = line.split(' ')
            for s in splits:
                if(s.startswith('/tr/ping.gif')):
                    return splitData(s, title)
    return None

def parseData(inputfile, outputfile, title):
    if(os.path.exists(outputfile)):
        os.remove(outputfile)
    with open(inputfile) as f:
        with open(outputfile,'w') as cf:
            f_csv = csv.writer(cf)
            f_csv.writerow(title)
            for line in f.readlines():
                data = splitLine(line, title)
                if(data!=None):
                    f_csv.writerow(data)


In [6]:
logfile = '_user_timo_logback_tr_ping_201806_20180603_10.142.8.63-ping.log.2018-06-03'
csvfile = '2018-06-03.csv'
title = ['device', 'app_version', 'device_id', 'os_version', 'page', 'tag', 'op','udid', 'stamp'] 

#解析数据
parseData(logfile, csvfile, title)

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv(csvfile)

df_deviceid = pd.read_csv('没有udid的device_id.csv')
df_deviceid_android = df_deviceid[(df_deviceid['device'] == 'android') & (df_deviceid['app_version']=='1.0.8')]
print('android 数据一共 %d 条' %len(df_deviceid_android))
deviceid_list = df_deviceid_android['device_id'].unique()
print(deviceid_list, len(deviceid_list))


android 数据一共 20 条
['1293e5233b3c0b50' '229e1acefa8f13c' '2db2ca9b27416509'
 '2f2bc22a2104f99e' '2f91e8613b2afaa5' '3c8d3cc04b5d6e53'
 '4680115f646e8de4' '5881faf943871328' '5bb04d965cf3391e'
 '6b6c9c7c5d95481a' '816a584ef289f800' '8cff71264c49d25d'
 '946dd0b960bb26fc' '9999c51c5b1d58e8' 'a42501c42a09f6e3'
 'aac72d3b5915dc1c' 'b9e437e3ed1c1d0e' 'd07f763469fabb60'
 'd7d107cbab862bdc' 'f3f74d6da734b27a'] 20


In [22]:
df_android = df[df['device']=='android']

for d in deviceid_list:
    print(df_android[df_android['device_id']==d][['device_id','tag','page','op','stamp']])
    print('-----------')

              device_id          tag     page    op                stamp
16223  1293e5233b3c0b50  app_startup  tr_init  auto  2018-06-03 13:40:09
-----------
            device_id          tag     page    op                stamp
1279  229e1acefa8f13c  app_startup  tr_init  auto  2018-06-03 00:19:37
-----------
             device_id          tag     page    op                stamp
1697  2db2ca9b27416509  app_startup  tr_init  auto  2018-06-03 01:37:37
1914  2db2ca9b27416509  app_startup  tr_init  auto  2018-06-03 01:49:00
-----------
             device_id                 tag               page     op  \
6932  2f2bc22a2104f99e         app_startup            tr_init   auto   
6933  2f2bc22a2104f99e   click_record_item            tr_home  click   
6954  2f2bc22a2104f99e   click_play_button  tr_edit_shorthand  click   
6982  2f2bc22a2104f99e   click_record_item            tr_home  click   
6990  2f2bc22a2104f99e   click_record_item            tr_home  click   
6995  2f2bc22a2104f99e  clic

In [24]:
deviceid = df_android['device_id'].unique()
print(deviceid, len(deviceid))

['821c332a57f02de0' 'ce27995259c31a4c' 'e0f24a73945ac1fe'
 '2f67e93e08f013d2' 'eee5987b1abe846c' 'd304b7e7917fd66b'
 'd982aa8401188457' 'add2c46ebd97e8fa' 'e831a21df5aa391e'
 '0d27f67e24a6c53e' '3bb6d87af00a30e9' 'c07cf23ca975c25a'
 '33d8bff4d2320663' '229e1acefa8f13c' '41f9d3c2c8336ff4'
 'dcaf0eb755be8ad4' '84cbea53fdf5a86d' 'a3485ab368154654'
 'e9764d3ca944319e' 'e3fe274894ca186e' '2db2ca9b27416509'
 '121067da5a9707b0' '00bb204198c9414f' '0308e058a39bebc6'
 '545c862ff867b805' '976364677bee7c02' '94f76a1a49b7b7a0'
 '06002af1d338af1f' '95cd9ba9f6b4eed2' '6f1bbed625c70e6b'
 '14011c2e9163fd50' '48d146e7ecb39de5' '35f128fe33e32330'
 '51c9fd57a1c20a0d' '086545e213784ec1' '1c2e2af0b38ec8a0'
 '29d8ef8eb4544697' '1192e22a8788f070' '8092f8cf23ef11af'
 '42b35840dfbd2dd5' 'f3f74d6da734b27a' '649adf6ae6fdff70'
 '6b255a5e03c6129d' '48ef3fb4621187ba' '51bda14f70cda07d'
 'ac39c053138b8e4e' '46846d7d9e9cdc00' 'be5c5b7f4d060d68'
 'd0c5a51dd488621c' '9939497cf7c17eb8' '683ee0e59af0f810'
 '8d20cb9fb26de

In [60]:
b = df_android[df_android['tag']=='app_startup']
c = b.groupby('device_id').count()
print(len(c))
c

37


,device,app_version,os_version,page,tag,op,udid
device_id,,,,,,,
01055ccee21ef064,1,1,1,1,1,1,0
04f66ebd5135e004,2,2,2,2,2,2,2
13b947651f6c4e2a,1,1,1,1,1,1,1
16ed7b7da15c5b63,1,1,1,1,1,1,1
1d318a6ea9bca34f,1,1,1,1,1,1,0
2114a6cc40d97281,1,1,1,1,1,1,0
32068429b9d6d841,1,1,1,1,1,1,0
38baec6441157651,3,3,3,3,3,3,0
3a5c553b131dcbd4,2,2,2,2,2,2,0


In [66]:
aa = df_android.groupby(['device_id','tag']).count()[['device']]
aa.columns=['count']
aa.head(20)

count
device_id        tag                                  
01055ccee21ef064 app_startup                         1
                 ble_data_upload                     1
                 click_play_button                   1
                 click_trpen_intro                   1
                 click_welcome_record_item           1
                 connection_successful               2
                 tr_ota_open                         1
                 tr_ota_update_click                 1
                 tr_ota_update_result                1
03c4a5d307891418 action_end_synctranslate            1
                 action_start_sync_record            4
                 action_start_synctranslate          1
                 action_successed_sync_record        5
                 action_successed_synctranslate      1
                 ble_data_download                   9
                 ble_data_upload                     1
                 click_app_setting                   1
                 click_home_nav                      2
                 click_pause_record_button           4
                 click_record_item                   5

In [73]:
device_list = pd.unique(df_android['device_id'])
device_list

array(['989081b4682bf3e9', '57c90c01e0cfc229', 'bc66a972873b0b03',
       '418cc0b62c45d29d', '272de72b07e3998f', '96a2674291bed3ee',
       'c098e239431e3753', 'bc6f3da1077b9572', '13a147ec92851666',
       '5033b8c997558187', '49bc29a4a99af67c', 'c1766ff2382b9f73',
       'bda322e56847e282', '3f9b892d962b865f', '95e064904286bfa7',
       '4220e5245592f0c2', 'f639c19f4b9db129', 'b63755a3b7e0d18c',
       '6ac6a19cada786b4', 'b829a3a976eddb0', '2f35842c4a77ac8d',
       '44be2cd8c9b4a8b', '89dc9516b194bfb5', '6382123522bf21b8',
       '35668b2ab4367fa1', '51c9fd57a1c20a0d', '4621080d73d7b774',
       '16ed7b7da15c5b63', '77dca973f11a040c', 'c4f08992794e2ffe',
       '5c9c8e9015104fd0', 'cccee4317ed29298', '9f06c0e9edfe2686',
       '07fa5fe10de24803', 'f5f05db81dbe7cfc', '493ce9d8c6511c53',
       '57a05aea65368afa', '32d610ed9b5cfccf', '41b31c9414e4e752',
       'dbe8cf58fc73bb8b', '13b947651f6c4e2a', '9de0cd68b0304c65',
       '12003bd91fe32436', 'b1274d5cbf5f966d', 'caa6e19b6858d2b1

In [81]:
bb = aa.loc['38baec6441157651']
'aa' in bb

False

In [82]:
rr = df_android.groupby(['device_id','udid']).count()

,,device,app_version,os_version,page,tag,op
device_id,udid,,,,,,
01055ccee21ef064,5.024040e+15,6,6,6,6,6,6
03c4a5d307891418,5.024040e+15,47,47,47,47,47,47
04f66ebd5135e004,5.024040e+15,131,131,131,131,131,131
061843681b1a25b6,5.024040e+15,12,12,12,12,12,12
07fa5fe10de24803,5.024040e+15,8,8,8,8,8,8
12003bd91fe32436,5.024040e+15,6,6,6,6,6,6
13a147ec92851666,5.024040e+15,78,78,78,78,78,78
13b947651f6c4e2a,5.024040e+15,156,156,156,156,156,156
1632fdd926556b26,5.024040e+15,10,10,10,10,10,10


In [120]:
for d1 in device_list:
    if('app_startup' not in aa.loc[d1]):
#         print(d1)
        df_r1 = df_android[df_android['device_id'] == d1]
        df_r2 = df_r1[df_r1['udid'].notnull()]
        if(len(df_r2)>0 and len(df_r1)>200):
            udid = df_r2.iloc[0]['udid']
            print("%.0f" %udid)
# d1 = 'bda322e56847e282'
# df_r1 = df_android[df_android['device_id'] == d1]
# df_r2 = df_r1[df_r1['udid'].notnull()]
# df_r2
# udid = df_r2.iloc[0]['udid']
# print("%s-%.0f" %(d1,udid))
        

5024040460069471
5020000000000020
5024040460072451
5024040460132861
5024040460024651
5024040460007651
5024040460020571
5024040460061101
5024040460068931
5024040460158871
5024040460002921
